In [1]:
# health_model_template.py

# ------------------------------
# 1. Import Libraries
# ------------------------------
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout
from tensorflow.keras.utils import to_categorical


In [ ]:
# ------------------------------
# 2. Load and Preprocess Data
# ------------------------------
def load_data(file_path):
    df = pd.read_csv(file_path)
    return df

def preprocess_data(df, target_column):
    # Encode categorical labels
    le = LabelEncoder()
    y = le.fit_transform(df[target_column])
    
    # Drop target and get features
    X = df.drop(columns=[target_column])
    
    # Scale features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    return X_scaled, y, le


In [ ]:

# ------------------------------
# 3. Define MLP Model
# ------------------------------
def build_mlp(input_dim, output_dim):
    model = Sequential()
    model.add(Dense(64, activation='relu', input_shape=(input_dim,)))
    model.add(Dropout(0.3))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(output_dim, activation='softmax'))
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model


In [ ]:

# ------------------------------
# 4. Define CNN Model (1D)
# ------------------------------
def build_cnn(input_shape, output_dim):
    model = Sequential()
    model.add(Conv1D(32, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(output_dim, activation='softmax'))
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model


In [ ]:
# ------------------------------
# 5. Train & Evaluate Model
# ------------------------------
def run_model(X, y, model_type="MLP"):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    if model_type == "MLP":
        model = build_mlp(input_dim=X.shape[1], output_dim=len(np.unique(y)))
    elif model_type == "CNN":
        # Reshape for CNN
        X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
        X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
        model = build_cnn(input_shape=X_train.shape[1:], output_dim=len(np.unique(y)))
    else:
        raise ValueError("Model type must be 'MLP' or 'CNN'")
    
    model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2, verbose=1)
    
    # Evaluation
    test_loss, test_acc = model.evaluate(X_test, y_test)
    print(f"Test Accuracy: {test_acc:.4f}")
    return model


In [ ]:

data_path = "your_preprocessed_data.csv"  
target_col = "recommended_treatment"      

df = load_data(data_path)
X, y, label_encoder = preprocess_data(df, target_col)

# Choose model type: "MLP" or "CNN"
final_model = run_model(X, y, model_type="MLP")
